### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
# import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

In [5]:
!ls

HW1.ipynb
data


В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [4]:
ratings = pd.read_csv('data/ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [7]:
movie_info = pd.read_csv('data/ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [8]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [9]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [10]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [11]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [16]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [11]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [12]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [13]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [14]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [15]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1892    Rain Man (1988)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [12]:
import torch
from torch.nn import functional as F

In [88]:
users = ratings["user_id"]
movies = ratings["movie_id"]
labels = ratings["rating"]
user_item = sp.coo_matrix((labels, (users, movies)))

In [89]:
torch_data = torch.sparse.FloatTensor(
    torch.as_tensor([user_item.row, user_item.col], dtype=torch.long), 
    torch.as_tensor(user_item.data), 
    torch.Size(user_item.shape)).to_dense()

In [242]:
implicit_torch_data = torch.sparse.FloatTensor(
    torch.as_tensor([user_item.row, user_item.col], dtype=torch.long), 
    torch.ones(user_item.data.shape), 
    torch.Size(user_item.shape)).to_dense()

In [224]:
movie_data = {}
for movie_id, movie_name, movie_category in zip(movie_info['movie_id'], 
                                                movie_info['name'], 
                                                movie_info['category']):
    movie_data[movie_id] = (movie_name, movie_category)

In [397]:
def cos_dist(x, y):
    norm = torch.sqrt(x.dot(x) * y.dot(y))
    norm[norm == 0] = 1
    return (1 - x.dot(y) / norm) / 2

In [398]:
def l2_dist(x, y):
    return torch.sum((x - y) ** 2)

In [399]:
def get_simmilar_movies(target, items_table, k=20, dist=cos_dist):
    dists = [(dist(target, vector), i) for i, vector in enumerate(items_table)]
    dists.sort()
    for d, i in dists[:k]:
        movie_name, movie_category = movie_data.get(i, ('Unknown', 'Unknown'))
        print(f'[d={d:.3f}] {movie_name} (Category: {movie_category})')

In [256]:
def get_recommendations(user, labels, items_table, k=20):
    predictions = torch.mm(items_table, user.unsqueeze(1)).squeeze()
    values, indexes = torch.sort(predictions, descending=True)
    for i in map(torch.Tensor.item, indexes):
        if k == 0:
            break
        if labels[i] != 0:
            continue
        k -= 1
        movie_name, movie_category = movie_data.get(i, ('Unknown', 'Unknown'))
        print(f'[d={predictions[i]:.3f}] {movie_name} (Category: {movie_category})')

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [389]:
class SGD:
    def __init__(self, users_dim, items_dim, hidden_dim, device='cuda'):
        self.users_embeddings = torch.rand(users_dim, hidden_dim, device=device) / np.sqrt(hidden_dim)
        self.items_embeddings = torch.rand(items_dim, hidden_dim, device=device) / np.sqrt(hidden_dim)
        self.device = device
        
    def predict(self):
        return torch.mm(self.users_embeddings, self.items_embeddings.transpose(0, 1))
    
    def fit(self, user_item, alpha, lr, n_epoch=1000, zeros_weight=1.0):
        user_item = user_item.to(self.device)
        for i in range(n_epoch):
            deltas = self.predict() - user_item
            if zeros_weight != 1.0:
                deltas[user_item == 0] *= zeros_weight
            users_grad = 2 * (torch.mm(deltas, self.items_embeddings) + alpha * self.users_embeddings)
            items_grad = 2 * (torch.mm(deltas.transpose(0, 1), self.users_embeddings) + alpha * self.items_embeddings)
            self.users_embeddings -= lr * users_grad
            self.items_embeddings -= lr * items_grad
        
    def to(self, device):
        self.users_embeddings = self.users_embeddings.to(device)
        self.items_embeddings = self.items_embeddings.to(device)
        self.device = device

In [390]:
n_users, n_items = torch_data.shape
model_sgd = SGD(n_users, n_items, 64)
model_sgd.fit(torch_data, 0.1, 0.0001, zeros_weight=0.01)

In [400]:
items_table = model_sgd.items_embeddings
get_simmilar_movies(items_table[1], items_table)

[d=0.000] Toy Story (1995) (Category: Animation|Children's|Comedy)
[d=0.047] Toy Story 2 (1999) (Category: Animation|Children's|Comedy)
[d=0.058] Bug's Life, A (1998) (Category: Animation|Children's|Comedy)
[d=0.119] Chicken Run (2000) (Category: Animation|Children's|Comedy)
[d=0.126] Ghostbusters (1984) (Category: Comedy|Horror)
[d=0.130] Raiders of the Lost Ark (1981) (Category: Action|Adventure)
[d=0.130] Shawshank Redemption, The (1994) (Category: Drama)
[d=0.130] Back to the Future (1985) (Category: Comedy|Sci-Fi)
[d=0.131] Aladdin (1992) (Category: Animation|Children's|Comedy|Musical)
[d=0.132] Rain Man (1988) (Category: Drama)
[d=0.136] Breakfast Club, The (1985) (Category: Comedy|Drama)
[d=0.140] Stand by Me (1986) (Category: Adventure|Comedy|Drama)
[d=0.140] Babe (1995) (Category: Children's|Comedy|Drama)
[d=0.141] Wrong Trousers, The (1993) (Category: Animation|Comedy)
[d=0.144] Sixth Sense, The (1999) (Category: Thriller)
[d=0.144] Big (1988) (Category: Comedy|Fantasy)
[d=0.

In [402]:
items_table = model_sgd.items_embeddings
get_simmilar_movies(items_table[1], items_table, dist=l2_dist)

[d=0.000] Toy Story (1995) (Category: Animation|Children's|Comedy)
[d=4.758] Toy Story 2 (1999) (Category: Animation|Children's|Comedy)
[d=5.066] Bug's Life, A (1998) (Category: Animation|Children's|Comedy)
[d=10.938] Aladdin (1992) (Category: Animation|Children's|Comedy|Musical)
[d=11.177] Ghostbusters (1984) (Category: Comedy|Horror)
[d=11.302] Chicken Run (2000) (Category: Animation|Children's|Comedy)
[d=11.854] Big (1988) (Category: Comedy|Fantasy)
[d=11.930] Breakfast Club, The (1985) (Category: Comedy|Drama)
[d=12.238] Rain Man (1988) (Category: Drama)
[d=12.457] Back to the Future (1985) (Category: Comedy|Sci-Fi)
[d=12.525] Lion King, The (1994) (Category: Animation|Children's|Musical)
[d=12.558] Shawshank Redemption, The (1994) (Category: Drama)
[d=12.648] Stand by Me (1986) (Category: Adventure|Comedy|Drama)
[d=12.974] Ferris Bueller's Day Off (1986) (Category: Comedy)
[d=12.982] Raiders of the Lost Ark (1981) (Category: Action|Adventure)
[d=13.058] Wrong Trousers, The (1993) 

In [403]:
get_recommendations(model_sgd.users_embeddings[4], torch_data[4], items_table)

[d=4.799] Terminator 2: Judgment Day (1991) (Category: Action|Sci-Fi|Thriller)
[d=4.635] Braveheart (1995) (Category: Action|Drama|War)
[d=4.508] Godfather, The (1972) (Category: Action|Crime|Drama)
[d=4.446] Matrix, The (1999) (Category: Action|Sci-Fi|Thriller)
[d=4.284] Fugitive, The (1993) (Category: Action|Thriller)
[d=4.070] Butch Cassidy and the Sundance Kid (1969) (Category: Action|Comedy|Western)
[d=3.973] Shawshank Redemption, The (1994) (Category: Drama)
[d=3.920] One Flew Over the Cuckoo's Nest (1975) (Category: Drama)
[d=3.903] Schindler's List (1993) (Category: Drama|War)
[d=3.841] Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954) (Category: Action|Drama)
[d=3.834] Back to the Future (1985) (Category: Comedy|Sci-Fi)
[d=3.809] Godfather: Part II, The (1974) (Category: Action|Crime|Drama)
[d=3.798] Princess Bride, The (1987) (Category: Action|Adventure|Comedy|Romance)
[d=3.744] Close Encounters of the Third Kind (1977) (Category: Drama|Sci-Fi)
[d=3.730] Hun

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [404]:
class ALS:
    def __init__(self, users_dim, items_dim, hidden_dim, device='cuda'):
        self.users_embeddings = torch.rand(users_dim, hidden_dim, device=device) / np.sqrt(hidden_dim)
        self.items_embeddings = torch.rand(items_dim, hidden_dim, device=device) / np.sqrt(hidden_dim)
        self.hidden_dim = hidden_dim
        self.device = device
        
    def predict(self):
        return torch.mm(self.users_embeddings, self.items_embeddings.transpose(0, 1))
    
    def fit(self, user_item, bound, alpha, n_epoch=10, non_zeros_weight=1.0):
        # Преобразуем в обозначения из http://yifanhu.net/PUB/cf.pdf
        user_item = user_item.to(self.device)
        C = 1 + non_zeros_weight * user_item
        P = (user_item >= bound).float()
        X = self.users_embeddings
        Xt = X.transpose(0, 1) 
        Y = self.items_embeddings
        Yt = Y.transpose(0, 1) 
        aI = alpha * torch.eye(self.hidden_dim, device=self.device)
        for _ in range(n_epoch):            
            for u in range(len(self.users_embeddings)):
                X[u] = torch.mm(torch.inverse(torch.mm(Yt * C[u], Y) + aI), Yt * C[u]).matmul(P[u])
            for i in range(len(self.items_embeddings)):
                Y[i] = torch.mm(torch.inverse(torch.mm(Xt * C[:, i], X) + aI), Xt * C[:, i]).matmul(P[:, i])
        
    def to(self, device):
        self.users_embeddings = self.users_embeddings.to(device)
        self.items_embeddings = self.items_embeddings.to(device)
        self.device = device

In [405]:
n_users, n_items = torch_data.shape
model_als = ALS(n_users, n_items, 64)
model_als.fit(torch_data, 4, 0.1, non_zeros_weight=5)

In [413]:
items_table = model_als.items_embeddings
get_simmilar_movies(items_table[1], items_table)

[d=0.000] Toy Story (1995) (Category: Animation|Children's|Comedy)
[d=0.020] Toy Story 2 (1999) (Category: Animation|Children's|Comedy)
[d=0.082] Shawshank Redemption, The (1994) (Category: Drama)
[d=0.083] Bug's Life, A (1998) (Category: Animation|Children's|Comedy)
[d=0.084] Raiders of the Lost Ark (1981) (Category: Action|Adventure)
[d=0.085] Sixth Sense, The (1999) (Category: Thriller)
[d=0.091] Usual Suspects, The (1995) (Category: Crime|Thriller)
[d=0.096] Schindler's List (1993) (Category: Drama|War)
[d=0.096] Princess Bride, The (1987) (Category: Action|Adventure|Comedy|Romance)
[d=0.097] Silence of the Lambs, The (1991) (Category: Drama|Thriller)
[d=0.100] Monty Python and the Holy Grail (1974) (Category: Comedy)
[d=0.100] Matrix, The (1999) (Category: Action|Sci-Fi|Thriller)
[d=0.101] Pulp Fiction (1994) (Category: Crime|Drama)
[d=0.102] Star Wars: Episode IV - A New Hope (1977) (Category: Action|Adventure|Fantasy|Sci-Fi)
[d=0.103] Pleasantville (1998) (Category: Comedy)
[d=0

In [414]:
items_table = model_als.items_embeddings
get_simmilar_movies(items_table[1], items_table, dist=l2_dist)

[d=0.000] Toy Story (1995) (Category: Animation|Children's|Comedy)
[d=0.073] Toy Story 2 (1999) (Category: Animation|Children's|Comedy)
[d=0.284] Shawshank Redemption, The (1994) (Category: Drama)
[d=0.297] Bug's Life, A (1998) (Category: Animation|Children's|Comedy)
[d=0.304] Raiders of the Lost Ark (1981) (Category: Action|Adventure)
[d=0.305] Usual Suspects, The (1995) (Category: Crime|Thriller)
[d=0.305] Sixth Sense, The (1999) (Category: Thriller)
[d=0.322] Princess Bride, The (1987) (Category: Action|Adventure|Comedy|Romance)
[d=0.333] Schindler's List (1993) (Category: Drama|War)
[d=0.335] Silence of the Lambs, The (1991) (Category: Drama|Thriller)
[d=0.341] Monty Python and the Holy Grail (1974) (Category: Comedy)
[d=0.342] Pulp Fiction (1994) (Category: Crime|Drama)
[d=0.344] Matrix, The (1999) (Category: Action|Sci-Fi|Thriller)
[d=0.351] Fugitive, The (1993) (Category: Action|Thriller)
[d=0.360] Christmas Story, A (1983) (Category: Comedy|Drama)
[d=0.360] Star Wars: Episode I

In [415]:
get_recommendations(model_als.users_embeddings[4], torch_data[4], items_table)

[d=0.905] Butch Cassidy and the Sundance Kid (1969) (Category: Action|Comedy|Western)
[d=0.815] Godfather, The (1972) (Category: Action|Crime|Drama)
[d=0.814] Terminator 2: Judgment Day (1991) (Category: Action|Sci-Fi|Thriller)
[d=0.783] Dirty Dozen, The (1967) (Category: Action|War)
[d=0.765] African Queen, The (1951) (Category: Action|Adventure|Romance|War)
[d=0.762] Aliens (1986) (Category: Action|Sci-Fi|Thriller|War)
[d=0.752] Ben-Hur (1959) (Category: Action|Adventure|Drama)
[d=0.741] Braveheart (1995) (Category: Action|Drama|War)
[d=0.725] Godfather: Part II, The (1974) (Category: Action|Crime|Drama)
[d=0.704] Matrix, The (1999) (Category: Action|Sci-Fi|Thriller)
[d=0.690] One Flew Over the Cuckoo's Nest (1975) (Category: Drama)
[d=0.648] French Connection, The (1971) (Category: Action|Crime|Drama|Thriller)
[d=0.607] Indiana Jones and the Last Crusade (1989) (Category: Action|Adventure)
[d=0.599] Schindler's List (1993) (Category: Drama|War)
[d=0.598] Boat, The (Das Boot) (1981) 

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [416]:
import numpy as np
from numpy import random
from sklearn.metrics import roc_auc_score

In [419]:
class BPR:
    def __init__(self, users_dim, items_dim, hidden_dim, device='cuda'):
        self.users_embeddings = torch.rand(users_dim, hidden_dim, device=device) / np.sqrt(hidden_dim)
        self.items_embeddings = torch.rand(items_dim, hidden_dim, device=device) / np.sqrt(hidden_dim)
        self.hidden_dim = hidden_dim
        self.device = device
        
    def predict(self):
        return torch.mm(self.users_embeddings, self.items_embeddings.transpose(0, 1))
    
    def fit(self, user_item, bound, alpha, lr, n_epoch=10000, batch_size=1000):
        n_users = len(self.users_embeddings)
        n_items = len(self.items_embeddings)
        
        implicit = (user_item >= bound)
        offset = implicit.sum(axis=1).cpu().numpy()
        indexes = implicit.float().argsort(axis=1, descending=True).cpu().numpy()
        
        def generate_pair():
            user = random.randint(0, n_users)
            while offset[user] == 0 or offset[user] == n_items:
                user = random.randint(0, n_users)
            positive_exaple = indexes[user][random.randint(0, offset[user])]
            negative_exaple = indexes[user][random.randint(offset[user], n_items)]
            return user, positive_exaple, negative_exaple
        
        for i in range(n_epoch):
            users_grad = alpha * self.users_embeddings
            items_grad = alpha * self.items_embeddings
            for _ in range(batch_size):
                user, positive, negative = generate_pair()
                delta_h = self.items_embeddings[positive] - self.items_embeddings[negative]
                delta_x = self.users_embeddings[user].dot(delta_h)
                sigma = 1 - torch.sigmoid(delta_x)
            
                users_grad[user] += sigma * delta_h
                items_grad[positive] += sigma * self.users_embeddings[user]
                items_grad[negative] += -sigma * self.users_embeddings[user]
            
            self.users_embeddings += lr * users_grad
            self.items_embeddings += lr * items_grad

In [420]:
n_users, n_items = torch_data.shape
model_bpr = BPR(n_users, n_items, 64)
model_bpr.fit(torch_data, 4, 0.01, 0.01, n_epoch=500, batch_size=1000)

In [421]:
print(f'ROC-AUC: {roc_auc_score((torch_data >= 4).flatten().cpu().numpy(), model_bpr.predict().flatten().cpu().numpy())}')

ROC-AUC: 0.8555851766539433


In [422]:
items_table = model_bpr.items_embeddings
get_simmilar_movies(items_table[1], items_table)

[d=0.000] Toy Story (1995) (Category: Animation|Children's|Comedy)
[d=0.003] Schindler's List (1993) (Category: Drama|War)
[d=0.003] Saving Private Ryan (1998) (Category: Action|Drama|War)
[d=0.004] Fargo (1996) (Category: Crime|Drama|Thriller)
[d=0.004] American Beauty (1999) (Category: Comedy|Drama)
[d=0.004] Raiders of the Lost Ark (1981) (Category: Action|Adventure)
[d=0.004] Silence of the Lambs, The (1991) (Category: Drama|Thriller)
[d=0.004] Jurassic Park (1993) (Category: Action|Adventure|Sci-Fi)
[d=0.004] Terminator 2: Judgment Day (1991) (Category: Action|Sci-Fi|Thriller)
[d=0.004] Star Wars: Episode VI - Return of the Jedi (1983) (Category: Action|Adventure|Romance|Sci-Fi|War)
[d=0.004] Shakespeare in Love (1998) (Category: Comedy|Romance)
[d=0.004] Matrix, The (1999) (Category: Action|Sci-Fi|Thriller)
[d=0.004] Braveheart (1995) (Category: Action|Drama|War)
[d=0.004] Star Wars: Episode IV - A New Hope (1977) (Category: Action|Adventure|Fantasy|Sci-Fi)
[d=0.004] Shawshank Re

In [423]:
items_table = model_bpr.items_embeddings
get_simmilar_movies(items_table[1], items_table, dist=l2_dist)

[d=0.000] Toy Story (1995) (Category: Animation|Children's|Comedy)
[d=0.183] Fugitive, The (1993) (Category: Action|Thriller)
[d=0.195] Ghostbusters (1984) (Category: Comedy|Horror)
[d=0.197] Groundhog Day (1993) (Category: Comedy|Romance)
[d=0.198] Men in Black (1997) (Category: Action|Adventure|Comedy|Sci-Fi)
[d=0.210] Toy Story 2 (1999) (Category: Animation|Children's|Comedy)
[d=0.216] Terminator, The (1984) (Category: Action|Sci-Fi|Thriller)
[d=0.219] Bug's Life, A (1998) (Category: Animation|Children's|Comedy)
[d=0.227] Godfather: Part II, The (1974) (Category: Action|Crime|Drama)
[d=0.240] E.T. the Extra-Terrestrial (1982) (Category: Children's|Drama|Fantasy|Sci-Fi)
[d=0.241] Stand by Me (1986) (Category: Adventure|Comedy|Drama)
[d=0.242] Usual Suspects, The (1995) (Category: Crime|Thriller)
[d=0.244] Pulp Fiction (1994) (Category: Crime|Drama)
[d=0.246] Alien (1979) (Category: Action|Horror|Sci-Fi|Thriller)
[d=0.251] Casablanca (1942) (Category: Drama|Romance|War)
[d=0.266] Blad

In [424]:
get_recommendations(model.users_embeddings[4], torch_data[4], items_table)

[d=2.295] American Beauty (1999) (Category: Comedy|Drama)
[d=1.915] Sixth Sense, The (1999) (Category: Thriller)
[d=1.898] Silence of the Lambs, The (1991) (Category: Drama|Thriller)
[d=1.780] Schindler's List (1993) (Category: Drama|War)
[d=1.779] Matrix, The (1999) (Category: Action|Sci-Fi|Thriller)
[d=1.768] Braveheart (1995) (Category: Action|Drama|War)
[d=1.766] Terminator 2: Judgment Day (1991) (Category: Action|Sci-Fi|Thriller)
[d=1.747] Godfather, The (1972) (Category: Action|Crime|Drama)
[d=1.745] Shawshank Redemption, The (1994) (Category: Drama)
[d=1.738] Being John Malkovich (1999) (Category: Comedy)
[d=1.737] Shakespeare in Love (1998) (Category: Comedy|Romance)
[d=1.730] Fargo (1996) (Category: Crime|Drama|Thriller)
[d=1.689] Princess Bride, The (1987) (Category: Action|Adventure|Comedy|Romance)
[d=1.663] L.A. Confidential (1997) (Category: Crime|Film-Noir|Mystery|Thriller)
[d=1.647] Back to the Future (1985) (Category: Comedy|Sci-Fi)
[d=1.641] Forrest Gump (1994) (Catego

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных